# 2: Написать программу, чтобы провести статистическую обработку текста

In [18]:
import math


def GetCharAmounts(__text:str, __char_len:int=1):
    _ret_char_amounts = dict()

    _index = 0
    while _index < len(__text) - __char_len + 1:
        _char = __text[_index:_index+__char_len]
        if _char not in _ret_char_amounts:
            _ret_char_amounts[_char] = 1
        else: _ret_char_amounts[_char] += 1
        _index+=1
    return _ret_char_amounts


def GetCharChances(__text:str, __char_len:int=1) -> dict[str, float]:
    return { _char: _amount / len(__text)
        for _char, _amount in GetCharAmounts(__text, __char_len).items()
    }


def GetEntropy(__text:str, __char_len:int=1) -> float:

    char_chances = { _char: _amount / len(__text)
        for _char, _amount in GetCharAmounts(__text, __char_len).items()
    }

    return -sum(
        _chance * math.log2(_chance)
        for _chance in char_chances.values()
    )


def MakeSchemeFano(__text:str, __char_len:int=1) -> list[str, float, str]:
    _scheme = sorted(
        map(
            lambda i: list(i) + [""],
            GetCharChances(__text, __char_len).items()
        ), 
        key=lambda i:-i[1]
    )
    _char_chances = list(map(lambda i: i[1], _scheme))

    def _MakeScheme(_scheme: list[str, float]):
        if len(_scheme) < 2: return

        _center_index = 1
        while sum(_char_chances[:_center_index]) < sum(_char_chances[_center_index:]): 
            _center_index += 1

        for _item in _scheme[:_center_index]:
            _item[2] += '0'
        _MakeScheme(_scheme[:_center_index])

        for _item in _scheme[_center_index:]:
            _item[2] += '1'
        _MakeScheme(_scheme[_center_index:])
    _MakeScheme(_scheme)
    return _scheme

def MakeSchemeHafman(__text:str, __char_len:int=1) -> list[str, float, str]:
    
    _scheme = sorted(
        map(
            lambda i: list(i) + [""],
            GetCharChances(__text, __char_len).items()
        ), 
        key=lambda i:-i[1]
    )

    _mapped_scheme = list(map(lambda i: [[i[0]]] + i[1:], _scheme))

    def _MakeScheme(_mapped_scheme: list[str, float]):
        if len(_mapped_scheme) <= 1: return
        _mapped_scheme = sorted(_mapped_scheme, key=lambda i:-i[1])

        # work with last
        for _index in range(len(_scheme)):
            if _scheme[_index][0] in _mapped_scheme[-1][0]: _scheme[_index][2] = '0' + _scheme[_index][2]
            if _scheme[_index][0] in _mapped_scheme[-2][0]: _scheme[_index][2] = '1' + _scheme[_index][2]

        # combine
        _popped_val = _mapped_scheme.pop()
        _mapped_scheme[-1][0] += _popped_val[0]
        _mapped_scheme[-1][1] += _popped_val[1]
        
        _MakeScheme(_mapped_scheme)
        
    _MakeScheme(_mapped_scheme)

    return _scheme



def GetAvrLen(__text:str, __char_len:int=1, __scheme:list[str, float, str]|None=None):
    if __scheme == None: __scheme = MakeScheme(__text, __char_len)
    return sum(map(lambda line: line[1] * len(line[2]), __scheme))


def Encode(text:str, scheme:list[str,float,str]):
    _ret_text = text
    for _item in scheme:
        _ret_text = _ret_text.replace(_item[0],_item[2])
    return _ret_text


def Decode(text:str, scheme:list[str,float,str]):
    if len(text) == 0 : return ''
    _index = 0
    while all(text[:_index] != _item[2] for _item in scheme):
        _index += 1
    return next(_item[0] for _item in scheme if text[:_index] == _item[2]) + Decode(text[_index:], scheme)


In [19]:
# 1. Определить энтропию, приходящуюся в среднем на одну букву, длину кода при равномерном кодировании и избыточность.

char_amounts = GetCharAmounts(DATA)
char_chances = GetCharChances(DATA)
code_entropy = GetEntropy(DATA)

# Определить длину кода
code_len = math.ceil(math.log2(len(char_amounts)))


# Определить избыточность
code_ext = 1 - (code_entropy / code_len)

print(code_entropy, code_len, code_ext)

1.8895782867502309 3 0.3701405710832564


In [20]:
# 2. Построить схему алфавитного кодирования для однобуквенных сочетаний методом Шеннона-Фано. 


MakeScheme(DATA)

[['a', 0.5789473684210527, '00'],
 ['h', 0.15789473684210525, '010'],
 ['j', 0.10526315789473684, '011'],
 ['d', 0.05263157894736842, '10'],
 ['f', 0.05263157894736842, '110'],
 ['g', 0.05263157894736842, '111']]

In [21]:
# 3. Найти среднюю длину элементарного кода, эффективность сжатия. Закодировать текст. Декодировать текст. 

scheme = MakeSchemeFano(DATA)

arv_len = sum(map(lambda line: line[1] * len(line[2]), scheme))

print(arv_len, code_entropy/arv_len)
text = DATA
text_encoded = Encode(text, scheme)
text_decoded = Decode(text_encoded, scheme)

print(text)
print(text_encoded)
print(text_decoded)

1.9473684210526314 0.9703239850879565
aaaaaaaaaaadfghhhjj
0000000000011101111011111101010110110
aaaaaaaaaaadfghhhjj


In [22]:
# 4. Построить схему алфавитного кодирования для двухбуквенных сочетаний методом Шеннона-Фано. Найти среднюю длину элементарного кода, эффективность сжатия, сравнить с результатами для однобуквенных сочетаний. Закодировать текст. Декодировать текст.

MakeSchemeFano(DATA,1)

[['a', 0.5789473684210527, '0'],
 ['h', 0.15789473684210525, '10'],
 ['j', 0.10526315789473684, '110'],
 ['d', 0.05263157894736842, '1110'],
 ['f', 0.05263157894736842, '11110'],
 ['g', 0.05263157894736842, '11111']]